In [10]:
feature_dict = {i:label for i,label in zip(
    range(4),
    ('sepal length in cm',
    'sepal width in cm',
    'petal length in cmd',
    'petal width in cmd'))}

import pandas as pd
import numpy as np

# df = pd.io.parsers.read_csv(
#     filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
#     header=None,
#     sep=','
#     )
df = pd.read_csv('iris.data')
df.columns = [l for i,l in sorted(feature_dict.items())]+["class label"]
df.head()

,sepal length in cm,sepal width in cm,petal length in cmd,petal width in cmd,class label
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [11]:
# cls_lab = {'Iris-setosa':1,'Iris-versicolor':2,'Iris-virginica':3}
# for i in np.arange(df.shape[0]):
#     if df.loc[i,'class label'] == 'Iris-setosa':
#         df.loc[i,'class label'] = 1
#     if df.loc[i,'class label'] == 'Iris-versicolor':
#         df.loc[i,'class label'] = 2
#     if df.loc[i,'class label'] == 'Iris-virginica':
#         df.loc[i,'class label'] = 3
# df.head()

from sklearn.preprocessing import LabelEncoder

X = df[['sepal length in cm','sepal width in cm','petal length in cmd','petal width in cmd']].values
y = df['class label']
df.head()

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1
print(label_encoder.classes_)


['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [12]:
# my mean
# feature_list = ['sepal length in cm','sepal width in cm','petal length in cmd','petal width in cmd']
# presur = df.pivot_table(index="class label", values=feature_list, aggfunc=np.mean)
# print(presur)

import numpy as np
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl],axis=0))
    print("class:{0} --> {1}".format(cl,mean_vectors[cl-1]))

class:1 --> [5.0041 3.4163 1.4653 0.2449]
class:2 --> [5.936 2.77  4.26  1.326]
class:3 --> [6.588 2.974 5.552 2.026]


In [17]:
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4),mean_vectors):
    class_sc_mat = np.zeros((4,4))
    for row in X[y==cl]:
        row,mv = row.reshape(4,1), mv.reshape(4,1)
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat
print(S_W)

[[38.9472 13.6751 24.6201  5.6598]
 [13.6751 17.0281  8.1254  4.9169]
 [24.6201  8.1254 27.2158  6.2507]
 [ 5.6598  4.9169  6.2507  6.1736]]


In [26]:
overall_mean = np.mean(X,axis=0)
S_B = np.zeros((4,4))
for i,mean_vec in enumerate(mean_vectors):
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1)
    overall_mean = overall_mean.reshape(4,1)
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)
print(S_B)

[[ 62.6649 -19.1924 163.3935  70.6115]
 [-19.1924  10.7842 -55.0015 -22.0477]
 [163.3935 -55.0015 431.0473 184.5397]
 [ 70.6115 -22.0477 184.5397  79.6021]]


In [32]:
eig_val,eig_vec = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
print(eig_val)
print(eig_vec)
print(eig_vec[:,1])
for i in range(len(eig_val)): 
    eigvc_sc = eig_vec[:,i].reshape(4,1)
    print('\nEigenvector {} :\n{}'.format(i+1,eigvc_sc.real))
    print('Eigenvalue {:} :{:.2e}'.format(i+1,eig_val[i].real))

[3.1875e+01+0.0000e+00j 2.7694e-01+0.0000e+00j 5.4881e-16+2.4416e-15j
 5.4881e-16-2.4416e-15j]
[[-0.2051+0.j     -0.0084+0.j      0.6276+0.j      0.6276-0.j    ]
 [-0.3869+0.j     -0.5891+0.j     -0.2636-0.2315j -0.2636+0.2315j]
 [ 0.5463+0.j      0.2545+0.j     -0.2581-0.2784j -0.2581+0.2784j]
 [ 0.714 +0.j     -0.7669+0.j     -0.0314+0.5813j -0.0314-0.5813j]]
[-0.0084+0.j -0.5891+0.j  0.2545+0.j -0.7669+0.j]

Eigenvector 1 :
[[-0.2051]
 [-0.3869]
 [ 0.5463]
 [ 0.714 ]]
Eigenvalue 1 :3.19e+01

Eigenvector 2 :
[[-0.0084]
 [-0.5891]
 [ 0.2545]
 [-0.7669]]
Eigenvalue 2 :2.77e-01

Eigenvector 3 :
[[ 0.6276]
 [-0.2636]
 [-0.2581]
 [-0.0314]]
Eigenvalue 3 :5.49e-16

Eigenvector 4 :
[[ 0.6276]
 [-0.2636]
 [-0.2581]
 [-0.0314]]
Eigenvalue 4 :5.49e-16


In [48]:
# eig_pairs = [(np.abs(eig_val),eig_vec[:,i]) for i in range(len(eig_val))]
# print(eig_pairs)
# eig_pairs = sorted(eig_pairs, key=lambda k:k[0], reverse=True)
# print(eig_pairs)

In [49]:
W = np.hstack((eig_pairs[0][1].reshape(4,1),eig_pairs[1][1].reshape(4,1)))
print(W)

[[-0.2051+0.j -0.0084+0.j]
 [-0.3869+0.j -0.5891+0.j]
 [ 0.5463+0.j  0.2545+0.j]
 [ 0.714 +0.j -0.7669+0.j]]


In [51]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

sklearn_lda = LDA(n_components=2)
X_lda_sklearn = sklearn_lda.fit_transform(X,y)

In [52]:
print(X_lda_sklearn)

[[-7.1825 -0.7467]
 [-7.5454 -0.2302]
 [-6.8735 -0.6338]
 [-8.1903  0.5456]
 [-7.7572  1.4834]
 [-7.2696  0.3834]
 [-7.6637  0.023 ]
 [-6.6191 -0.9768]
 [-7.4037 -0.9043]
 [-8.4539  0.6798]
 [-7.2822 -0.0759]
 [-7.3855 -1.0294]
 [-7.6301 -0.7662]
 [-9.8979  1.6151]
 [-9.2101  2.7525]
 [-8.6289  1.8546]
 [-7.833   0.6135]
 [-8.1331  0.9979]
 [-8.0781  1.1653]
 [-7.5551 -0.1504]
 [-7.639   1.2301]
 [-8.7347  0.9047]
 [-6.3011  0.4647]
 [-6.6285 -0.3543]
 [-6.8285 -0.9293]
 [-6.8763  0.4896]
 [-7.9817  0.244 ]
 [-8.0452  0.1219]
 [-6.8917 -0.5087]
 [-6.8192 -0.7205]
 [-7.4214  0.5946]
 [-9.1924  1.2535]
 [-9.5253  1.85  ]
 [-7.4037 -0.9043]
 [-8.0088 -0.1283]
 [-8.6629  0.4388]
 [-7.4037 -0.9043]
 [-6.9914 -0.6691]
 [-7.7455  0.0261]
 [-7.9691  0.7032]
 [-5.7081 -1.8904]
 [-7.3    -0.2394]
 [-6.4611  1.2639]
 [-6.9217  1.0737]
 [-6.8159 -0.47  ]
 [-8.1449  0.7927]
 [-7.2457 -0.3261]
 [-8.3721  0.6768]
 [-7.7273 -0.099 ]
 [ 1.3988  0.0406]
 [ 1.7386  0.4877]
 [ 2.3556 -0.0832]
 [ 2.2048 -1